In [ ]:
# Figure 3 Cell-free biomarker performance as a panel in a test cohort 
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap

from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    roc_auc_score, roc_curve, confusion_matrix,
    recall_score, make_scorer
)
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# ------------------------------ CONFIG ------------------------------ #
DATA_DIR   = "/Users/suraj/Documents/Supernatant_Cyto_Data"
COMBINED   = f"{DATA_DIR}/combinedELISA2.csv"
PROSPECT   = f"{DATA_DIR}/prospectiveELISA2.csv"

TARGET_COL = "Barretts"
BIOMARKERS = ["TFF3", "REG4", "MEP1A", "SPINK4", "FABP2", "CXCL8"]

N_SPLITS    = 5
RANDOM_SEED = 42

# ------------------------------ SCORERS ------------------------------ #
def specificity_score(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    return tn / (tn + fp) if (tn + fp) > 0 else 0.0

specificity_scorer = make_scorer(specificity_score)
sensitivity_scorer = make_scorer(recall_score, pos_label=1)

# ------------------------------ DATA ------------------------------ #
def load_subset(path):
    df = pd.read_csv(path)
    return df[[TARGET_COL] + BIOMARKERS].copy()

combined    = load_subset(COMBINED)
prospective = load_subset(PROSPECT)

full_df = pd.concat([combined, prospective], ignore_index=True)
X = full_df[BIOMARKERS]
y = full_df[TARGET_COL].astype(int)

print("\n=== Dataset Info ===")
n = len(full_df); pos = int(full_df[TARGET_COL].sum()); neg = n - pos
print(f"Full Dataset: N={n} | Barretts={pos} | Non-Barretts={neg}")

# ------------------------------ PIPELINE ------------------------------ #
def make_pipeline(model):
    num_pipe = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    pre = ColumnTransformer([
        ('num', num_pipe, BIOMARKERS)
    ])
    return Pipeline([('pre', pre), ('model', model)])

cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_SEED)
scoring = {"roc_auc": "roc_auc", "sensitivity": sensitivity_scorer, "specificity": specificity_scorer}

pos_weight = (y == 0).sum() / (y == 1).sum()

models = {
    "LogReg": LogisticRegression(max_iter=10000, class_weight='balanced', solver='liblinear'),
    "Lasso": LogisticRegressionCV(
        Cs=10, cv=5, penalty='l1', solver='saga', scoring='roc_auc',
        class_weight='balanced', max_iter=10000, random_state=RANDOM_SEED
    ),
    "RandomForest": RandomForestClassifier(n_estimators=500, class_weight='balanced', random_state=RANDOM_SEED),
    "XGBoost": XGBClassifier(
        n_estimators=300, learning_rate=0.05, max_depth=4,
        subsample=0.9, colsample_bytree=0.9, scale_pos_weight=pos_weight,
        eval_metric='logloss', use_label_encoder=False, random_state=RANDOM_SEED
    ),
    "LightGBM": LGBMClassifier(
        n_estimators=300, learning_rate=0.05, subsample=0.9,
        colsample_bytree=0.9, class_weight='balanced',
        random_state=RANDOM_SEED, verbosity=-1
    )
}

# ------------------------------ TRAIN + EVAL ------------------------------ #
all_results = []
cv_roc_curves = {model: [] for model in models}
for name, base_model in models.items():
    print(f"\n=== {name} ===")
    pipe = make_pipeline(base_model)
    fold_probs = []
    fold_true = []
    scores = cross_validate(pipe, X, y, cv=cv, scoring=scoring, n_jobs=-1, return_estimator=True)

    for train_idx, test_idx in cv.split(X, y):
        pipe.fit(X.iloc[train_idx], y.iloc[train_idx])
        probs = pipe.predict_proba(X.iloc[test_idx])[:, 1]
        fold_probs.extend(probs)
        fold_true.extend(y.iloc[test_idx])

    pipe.fit(X, y)
    all_results.append({
        "name": name, "pipeline": pipe,
        "cv_auc_mean": scores['test_roc_auc'].mean(),
        "cv_auc_sd": scores['test_roc_auc'].std(),
        "cv_sens_mean": scores['test_sensitivity'].mean(),
        "cv_sens_sd": scores['test_sensitivity'].std(),
        "cv_spec_mean": scores['test_specificity'].mean(),
        "cv_spec_sd": scores['test_specificity'].std(),
        "cv_probs": np.array(fold_probs),
        "cv_true": np.array(fold_true)
    })

# ------------------------------ ROC CURVES (CV) ------------------------------ #
def roc_cv_panel(results):
    plt.figure(figsize=(6,5))
    desired_order = ["LightGBM", "XGBoost", "RandomForest", "LogReg", "Lasso"]
    results_sorted = sorted(results, key=lambda x: desired_order.index(x["name"]) if x["name"] in desired_order else 99)
    for r in results_sorted:
        fpr, tpr, _ = roc_curve(r["cv_true"], r["cv_probs"])
        auc = roc_auc_score(r["cv_true"], r["cv_probs"])
        plt.plot(fpr, tpr, label=f"{r['name']} (CV AUC={auc:.2f})")
    plt.plot([0,1],[0,1],'k--')
    plt.title("Cross-Validated ROC Curves")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend()
    plt.tight_layout()
    plt.show()

roc_cv_panel(all_results)

# ------------------------------ SHAP ANALYSIS FOR LIGHTGBM ------------------------------ #
lgb_res = next((r for r in all_results if r['name'] == 'LightGBM'), None)
if lgb_res is not None:
    model = lgb_res['pipeline'].named_steps['model']
    X_proc = lgb_res['pipeline'].named_steps['pre'].transform(X)
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_proc)
    if isinstance(shap_values, list) and len(shap_values) > 1:
        shap_values = shap_values[1]

    shap.summary_plot(shap_values, X_proc, feature_names=BIOMARKERS, plot_type='bar')
    shap.summary_plot(shap_values, X_proc, feature_names=BIOMARKERS)
    shap.dependence_plot("CXCL8", shap_values, X_proc, feature_names=BIOMARKERS)

# ------------------------------ PERFORMANCE TABLE ------------------------------ #
perf_rows = []
for r in all_results:
    perf_rows.append({
        "Model": r["name"],
        "CV AUC (mean±sd)": f"{r['cv_auc_mean']:.3f} ± {r['cv_auc_sd']:.3f}",
        "CV Sens (mean±sd)": f"{r['cv_sens_mean']:.2f} ± {r['cv_sens_sd']:.2f}",
        "CV Spec (mean±sd)": f"{r['cv_spec_mean']:.2f} ± {r['cv_spec_sd']:.2f}"
    })
perf_df = pd.DataFrame(perf_rows).sort_values("CV AUC (mean±sd)", ascending=False)
print("\nPerformance Table:")
print(perf_df.to_string(index=False))

In [ ]:
# Figure 4 Secretory protein panel performance in an external prospective test cohort. 


import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import textwrap

from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    roc_auc_score, roc_curve, confusion_matrix,
    recall_score, make_scorer
)
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# ------------------------------ CONFIG ------------------------------ #
DATA_DIR   = "/Users/suraj/Documents/Supernatant_Cyto_Data"
COMBINED   = f"{DATA_DIR}/combinedELISA2.csv"
PROSPECT   = f"{DATA_DIR}/prospectiveELISA2.csv"
EXTERNAL   = f"{DATA_DIR}/prospective_Final_2.csv"

TARGET_COL = "Barretts"
BIOMARKERS = ["TFF3", "REG4", "MEP1A", "SPINK4", "FABP2", "CXCL8"]

N_SPLITS    = 5
RANDOM_SEED = 42

# ------------------------------ SCORERS ------------------------------ #
def specificity_score(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    return tn / (tn + fp) if (tn + fp) > 0 else 0.0

specificity_scorer = make_scorer(specificity_score)
sensitivity_scorer = make_scorer(recall_score, pos_label=1)

# ------------------------------ DATA ------------------------------ #
def load_subset(path):
    df = pd.read_csv(path)
    return df[[TARGET_COL] + BIOMARKERS].copy()

combined    = load_subset(COMBINED)
prospective = load_subset(PROSPECT)
external    = load_subset(EXTERNAL)

dev_df = pd.concat([combined, prospective], ignore_index=True)
X_dev = dev_df[BIOMARKERS]
y_dev = dev_df[TARGET_COL].astype(int)
X_ext = external[BIOMARKERS]
y_ext = external[TARGET_COL].astype(int)

print("\n=== Cohort counts ===")
for name, df in [("Development (CV)", dev_df), ("External (hold-out)", external)]:
    n = len(df); pos = int(df[TARGET_COL].sum()); neg = n - pos
    print(f"{name:22s}  N={n:4d} | Barretts={pos:3d} | Non-Barretts={neg:3d}")

# ------------------------------ PIPELINE ------------------------------ #
def make_pipeline(model):
    num_pipe = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    pre = ColumnTransformer([
        ('num', num_pipe, BIOMARKERS)
    ])
    return Pipeline([('pre', pre), ('model', model)])

cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_SEED)
scoring = {"roc_auc": "roc_auc", "sensitivity": sensitivity_scorer, "specificity": specificity_scorer}

pos_weight = (y_dev == 0).sum() / (y_dev == 1).sum()

models = {
    "LogReg": LogisticRegression(max_iter=10000, class_weight='balanced', solver='liblinear'),
    "Lasso": LogisticRegressionCV(
        Cs=10, cv=5, penalty='l1', solver='saga', scoring='roc_auc',
        class_weight='balanced', max_iter=10000, random_state=RANDOM_SEED
    ),
    "RandomForest": RandomForestClassifier(n_estimators=500, class_weight='balanced', random_state=RANDOM_SEED),
    "XGBoost": XGBClassifier(
        n_estimators=300, learning_rate=0.05, max_depth=4,
        subsample=0.9, colsample_bytree=0.9, scale_pos_weight=pos_weight,
        eval_metric='logloss', use_label_encoder=False, random_state=RANDOM_SEED
    ),
    "LightGBM": LGBMClassifier(
        n_estimators=300, learning_rate=0.05, subsample=0.9,
        colsample_bytree=0.9, class_weight='balanced',
        random_state=RANDOM_SEED, verbosity=-1  # suppress warning
    )
}

# ------------------------------ TRAIN + EVAL ------------------------------ #
def evaluate_on_external(pipe, X_ext, y_ext):
    probs = pipe.predict_proba(X_ext)[:, 1]
    preds = (probs >= 0.5).astype(int)
    auc = roc_auc_score(y_ext, probs)
    sens = recall_score(y_ext, preds)
    spec = specificity_score(y_ext, preds)
    cm = confusion_matrix(y_ext, preds)
    return dict(auc=auc, sens=sens, spec=spec, probs=probs, preds=preds, cm=cm)

all_results = []
for name, base_model in models.items():
    print(f"\n=== {name} ===")
    pipe = make_pipeline(base_model)
    scores = cross_validate(pipe, X_dev, y_dev, cv=cv, scoring=scoring, n_jobs=-1)

    pipe.fit(X_dev, y_dev)
    ext_metrics = evaluate_on_external(pipe, X_ext, y_ext)

    all_results.append({
        "name": name, "pipeline": pipe,
        "cv_auc_mean": scores['test_roc_auc'].mean(),
        "cv_auc_sd": scores['test_roc_auc'].std(),
        "cv_sens_mean": scores['test_sensitivity'].mean(),
        "cv_sens_sd": scores['test_sensitivity'].std(),
        "cv_spec_mean": scores['test_specificity'].mean(),
        "cv_spec_sd": scores['test_specificity'].std(),
        "external": ext_metrics
    })

# ------------------------------ ROC PLOT ------------------------------ #
plt.figure(figsize=(6,5))
for r in all_results:
    fpr, tpr, _ = roc_curve(y_ext, r['external']['probs'])
    plt.plot(fpr, tpr, label=f"{r['name']} (AUC={r['external']['auc']:.2f})")
plt.plot([0,1],[0,1],'k--')
plt.title("External Validation ROC Curves")
plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
plt.legend(); plt.tight_layout(); plt.show()

# ------------------------------ SHAP ANALYSIS FOR LIGHTGBM ------------------------------ #
lgb_res = next((r for r in all_results if r['name'] == 'LightGBM'), None)
if lgb_res is not None:
    model = lgb_res['pipeline'].named_steps['model']
    X_proc = lgb_res['pipeline'].named_steps['pre'].transform(X_dev)
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_proc)
    if isinstance(shap_values, list) and len(shap_values) > 1:
        shap_values = shap_values[1]

    shap.summary_plot(shap_values, X_proc, feature_names=BIOMARKERS, plot_type='bar')
    shap.summary_plot(shap_values, X_proc, feature_names=BIOMARKERS)

    # Optional: investigate CXCL8
    shap.dependence_plot("CXCL8", shap_values, X_proc, feature_names=BIOMARKERS)

# ------------------------------ CONFUSION MATRIX ------------------------------ #
    cm = lgb_res['external']['cm']
    plt.figure(figsize=(4,3))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Non-Barretts','Barretts'], yticklabels=['Non-Barretts','Barretts'])
    plt.title('External Confusion Matrix - LightGBM')
    plt.xlabel('Predicted'); plt.ylabel('True')
    plt.tight_layout(); plt.show()

# ------------------------------ PERFORMANCE TABLE ------------------------------ #
perf_rows = []
for r in all_results:
    perf_rows.append({
        "Model": r["name"],
        "CV AUC (mean±sd)": f"{r['cv_auc_mean']:.3f} ± {r['cv_auc_sd']:.3f}",
        "CV Sens (mean±sd)": f"{r['cv_sens_mean']:.2f} ± {r['cv_sens_sd']:.2f}",
        "CV Spec (mean±sd)": f"{r['cv_spec_mean']:.2f} ± {r['cv_spec_sd']:.2f}",
        "External AUC": f"{r['external']['auc']:.3f}",
        "External Sens/Spec": f"Sens {r['external']['sens']:.2f} | Spec {r['external']['spec']:.2f}"
    })
perf_df = pd.DataFrame(perf_rows).sort_values("External AUC", ascending=False)
print("\nPerformance Table:")
print(perf_df.to_string(index=False))

In [ ]:
# Figure 5 Reduced secretory protein panel performance on a more sensitive assay platform. 


import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shap

from sklearn.model_selection import StratifiedKFold, cross_validate, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    roc_auc_score, roc_curve, recall_score,
    confusion_matrix, ConfusionMatrixDisplay, make_scorer
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample


# ------------------------------ CONFIG ------------------------------ #
DATA_PATH  = "/Users/suraj/Documents/Supernatant_Cyto_Data/Dec 2025/All ELISA cohorts.csv"
TARGET_COL = "Barretts"

ALL_MARKERS = [
    "TFF3",
    "MEP1A",
    "SPINK4",
    "CXCL8",
    "Simoa REG4",
    "Simoa FABP2",
    "Simoa TFF3"
]

N_SPLITS    = 5
RANDOM_SEED = 42
BOOTSTRAPS  = 2000


# ------------------------------ DATA ------------------------------ #
df = pd.read_csv(DATA_PATH)
df[TARGET_COL] = df[TARGET_COL].astype(int)
df = df[[TARGET_COL] + ALL_MARKERS].dropna().copy()

X_full = df[ALL_MARKERS]
y_full = df[TARGET_COL].values

print(f"\nLoaded dataset: N={len(df)}, BE={y_full.sum()}, Controls={len(df)-y_full.sum()}")


# ------------------------------ PIPELINE ------------------------------ #
def make_pipeline(model, feats):
    num_pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])
    pre = ColumnTransformer(
        [("num", num_pipe, feats)],
        sparse_threshold=0.0
    )
    return Pipeline([("pre", pre), ("model", model)])


# ------------------------------ METRICS ------------------------------ #
def specificity_score(y_true, y_pred):
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    return tn / (tn + fp) if (tn + fp) > 0 else 0.0

specificity_scorer = make_scorer(specificity_score)
sensitivity_scorer = make_scorer(recall_score, pos_label=1)


# ------------------------------ BOOTSTRAP CI ------------------------------ #
def bootstrap_ci(y_true, y_hat, metric_fn, n_boot=BOOTSTRAPS, alpha=0.95):
    rng = np.random.RandomState(42)
    stats = []

    for _ in range(n_boot):
        idx = resample(np.arange(len(y_true)), random_state=rng)
        try:
            stats.append(metric_fn(y_true[idx], y_hat[idx]))
        except Exception:
            pass

    lower = np.percentile(stats, (1 - alpha) / 2 * 100)
    upper = np.percentile(stats, (1 + alpha) / 2 * 100)
    return lower, upper


# ------------------------------ FEATURE RANKING ------------------------------ #
rf_base = RandomForestClassifier(
    n_estimators=500,
    class_weight="balanced",
    random_state=RANDOM_SEED,
    n_jobs=-1
)

rank_pipe = make_pipeline(rf_base, ALL_MARKERS)
rank_pipe.fit(X_full, y_full)

importances = pd.Series(
    rank_pipe.named_steps["model"].feature_importances_,
    index=ALL_MARKERS
).sort_values(ascending=False)

feature_order = list(importances.index)

print("\nFeature importances:")
print(importances)


# ------------------------------ DROP-TEST ------------------------------ #
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_SEED)
results = []

plt.figure(figsize=(7, 6))

for k in range(len(feature_order), 1, -1):
    feats = feature_order[:k]
    pipe = make_pipeline(rf_base, feats)

    oof_probs = cross_val_predict(
        pipe, X_full[feats], y_full,
        cv=cv, method="predict_proba", n_jobs=-1
    )[:, 1]

    oof_pred = (oof_probs >= 0.5).astype(int)

    auc = roc_auc_score(y_full, oof_probs)
    sens = recall_score(y_full, oof_pred)
    spec = specificity_score(y_full, oof_pred)

    auc_ci = bootstrap_ci(y_full, oof_probs, roc_auc_score)
    sens_ci = bootstrap_ci(y_full, oof_pred, lambda yt, yp: recall_score(yt, yp))
    spec_ci = bootstrap_ci(y_full, oof_pred, specificity_score)

    fpr, tpr, _ = roc_curve(y_full, oof_probs)
    plt.plot(fpr, tpr, label=f"Top-{k} ({', '.join(feats)}) (AUC={auc:.2f})")

    results.append({
        "k": k,
        "Features": ", ".join(feats),
        "OOF AUC (95% CI)": f"{auc:.3f} [{auc_ci[0]:.3f}–{auc_ci[1]:.3f}]",
        "Sens @0.5 (95% CI)": f"{sens:.2f} [{sens_ci[0]:.2f}–{sens_ci[1]:.2f}]",
        "Spec @0.5 (95% CI)": f"{spec:.2f} [{spec_ci[0]:.2f}–{spec_ci[1]:.2f}]"
    })

plt.plot([0,1],[0,1],'k--')
plt.title("ROC Curves — Drop Test (Random Forest)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(fontsize=8)
plt.tight_layout()
plt.show()


# ------------------------------ CONFUSION MATRIX (TOP-2) ------------------------------ #
top2_feats = feature_order[:2]
print(f"\nPlotting confusion matrix for Top-2 markers: {top2_feats}")

pipe_top2 = make_pipeline(rf_base, top2_feats)

oof_probs_top2 = cross_val_predict(
    pipe_top2, X_full[top2_feats], y_full,
    cv=cv, method="predict_proba", n_jobs=-1
)[:, 1]

oof_pred_top2 = (oof_probs_top2 >= 0.5).astype(int)

cm = confusion_matrix(y_full, oof_pred_top2)
disp = ConfusionMatrixDisplay(cm, display_labels=["Control", "BE"])

disp.plot(cmap="Blues", values_format="d")
plt.title(f"Confusion Matrix — Top-2 ({', '.join(top2_feats)})")
plt.tight_layout()
plt.show()


# ------------------------------ SUMMARY TABLE ------------------------------ #
summary_df = pd.DataFrame(results)
print("\n=== Summary (OOF metrics with 95% CI) ===")
print(summary_df.to_string(index=False))

